In [ ]:
#Check GPU
!nvidia-smi


In [ ]:
#Install PyTorch and core dependencies
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install einops timm matplotlib opencv-python scipy


In [ ]:
#Clone Mast3r repository
!git clone https://github.com/naver/mast3r.git
%cd mast3r
!git submodule update --init --recursive


In [ ]:
#Verify DUSt3R submodule
!ls dust3r


In [ ]:
#Install Mast3r and DUSt3R requirements
!pip install -r dust3r/requirements.txt
!pip install -r requirements.txt


In [ ]:
#Optional CUDA extension for CroCo (RoPE2D)

%cd /content/mast3r/dust3r/croco/models/curope
!python setup.py build_ext --inplace
%cd /content/mast3r


In [ ]:
#Create image input directory
import os
os.makedirs("images", exist_ok=True)


In [ ]:
#Upload images to Colab
from google.colab import files
uploaded = files.upload()


In [ ]:
#Move uploaded images into images folder
import shutil
for fname in uploaded.keys():
    shutil.move(fname, "images/" + fname)


In [ ]:
#Create output directory
import os
os.makedirs("/content/mast3r/output", exist_ok=True)
#Import Mast3r and DUSt3R modules
import mast3r.utils.path_to_dust3r  # enables dust3r imports

from dust3r.inference import inference
from dust3r.model import AsymmetricCroCo3DStereo
from dust3r.utils.image import load_images
from dust3r.image_pairs import make_pairs
from dust3r.cloud_opt import global_aligner, GlobalAlignerMode

import numpy as np
import torch
#Load pretrained DUSt3R model
device = "cuda"

print("Loading model…")
model = AsymmetricCroCo3DStereo.from_pretrained(
    "naver/DUSt3R_ViTLarge_BaseDecoder_512_dpt"
).to(device)

print("Loading images…")
#Load input images
images = load_images("./images", size=320)
print("Loaded", len(images), "images")

print("Making pairs…")
#Create image pairs
pairs = make_pairs(images, scene_graph="complete", symmetrize=True)

print("Running inference…")
#Run DUSt3R inference
output = inference(pairs, model, device, batch_size=1)

print("Global alignment…")
#Global alignment of the scene
scene = global_aligner(output, device=device, mode=GlobalAlignerMode.PointCloudOptimizer)
scene.compute_global_alignment(init="mst", niter=300, schedule="cosine", lr=0.01)

print("Done computing scene!")


In [ ]:
# ===== CELL: export Mast3r-style fused PLY from `scene` =====

import os
import numpy as np
import torch

os.makedirs("/content/mast3r/output", exist_ok=True)

# images as numpy, shape per image HxWx3
#Export Mast3r-style fused sparse PLY
imgs = np.array(scene.imgs)

# Mast3r/DUSt3R style: use sparse pts3d + masks
pts3d_list = scene.get_pts3d()    # list of HxWx3 tensors
masks_list = scene.get_masks()    # list of HxW tensors

pts_all = []
cols_all = []

for img, pts, msk in zip(imgs, pts3d_list, masks_list):
    pts_np = pts.detach().cpu().numpy()       # HxWx3
    msk_np = msk.detach().cpu().numpy() > 0   # HxW

    # keep only valid + finite points
    valid = msk_np & np.isfinite(pts_np.sum(axis=-1))
    pts_all.append(pts_np[valid])
    cols_all.append(img[valid])

pts_all = np.concatenate(pts_all, axis=0)   # (N, 3)
cols_all = np.concatenate(cols_all, axis=0) # (N, 3)

print("Total points in fused cloud:", pts_all.shape[0])

ply_path = "/content/mast3r/output/scene_mast3r_sparse.ply"

with open(ply_path, "w") as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write(f"element vertex {pts_all.shape[0]}\n")
    f.write("property float x\n")
    f.write("property float y\n")
    f.write("property float z\n")
    f.write("property uchar red\n")
    f.write("property uchar green\n")
    f.write("property uchar blue\n")
    f.write("end_header\n")
    for (x, y, z), (r, g, b) in zip(pts_all, cols_all):
        f.write(f"{x} {y} {z} {int(r)} {int(g)} {int(b)}\n")

print("Saved PLY:", ply_path)
#Download fused PLY file
from google.colab import files

files.download(ply_path)


In [ ]:
!ls -lh /content/mast3r/output


In [ ]:
# ======= VIEW ANY .PLY FILE IN COLAB (scene_mast3r_sparse.ply) =======
#View PLY using Open3D and Plotly
!pip install -q open3d plotly

from google.colab import files
import open3d as o3d
import numpy as np
import plotly.graph_objects as go

print("Upload the PLY file (scene_mast3r_sparse.ply)...")
uploaded = files.upload()

if not uploaded:
    print("No file uploaded.")
else:
    fname = next(iter(uploaded.keys()))
    print("Loaded:", fname)

    # Load point cloud using Open3D
    pcd = o3d.io.read_point_cloud(fname)

    pts = np.asarray(pcd.points)

    if pts.size == 0:
        print("No points found. The file might be empty or corrupted.")
    else:
        x, y, z = pts[:, 0], pts[:, 1], pts[:, 2]

        # Colors (if present)
        if pcd.has_colors():
            cols = np.asarray(pcd.colors)
            col_strings = [
                f"rgb({int(r*255)}, {int(g*255)}, {int(b*255)})"
                for r, g, b in cols
            ]
            marker_kwargs = dict(size=2, color=col_strings)
        else:
            marker_kwargs = dict(size=2, color="black")

        fig = go.Figure(
            data=[go.Scatter3d(
                x=x,
                y=y,
                z=z,
                mode="markers",
                marker=marker_kwargs,
            )]
        )

        fig.update_layout(
            width=900,
            height=900,
            scene=dict(
                xaxis_title="X",
                yaxis_title="Y",
                zaxis_title="Z",
                aspectmode="data",
            ),
            title=f"Visualization of {fname}",
        )

        fig.show()


In [ ]:
%cd /content/mast3r

import os
import numpy as np
import torch

os.makedirs("/content/mast3r/output", exist_ok=True)

# get 3D points
#Export alternative colored PLY
pts3d = scene.get_pts3d()[0].detach().cpu().numpy()
if pts3d.shape[1] > 3:
    pts3d = pts3d[:, :3]

# get first image as RGB uint8
img0 = scene.imgs[0]
arr = img0.detach().cpu().numpy() if isinstance(img0, torch.Tensor) else np.array(img0)

# CHW → HWC
if arr.ndim == 3 and arr.shape[0] in (1, 3, 4):
    arr = np.transpose(arr, (1, 2, 0))

# scale to 0–255
if arr.max() <= 1:
    arr = (arr * 255).clip(0, 255)

img_uint8 = arr.astype(np.uint8)
if img_uint8.shape[2] > 3:
    img_uint8 = img_uint8[:, :, :3]  # drop alpha

H, W, _ = img_uint8.shape
pixels = img_uint8.reshape(-1, 3)

repeat_factor = int(np.ceil(pts3d.shape[0] / pixels.shape[0]))
colors = np.tile(pixels, (repeat_factor, 1))[:pts3d.shape[0]]
colors = colors[:, :3]

ply_path = "/content/mast3r/output/scene_color.ply"

with open(ply_path, "w") as f:
    f.write("ply\nformat ascii 1.0\n")
    f.write(f"element vertex {pts3d.shape[0]}\n")
    f.write("property float x\nproperty float y\nproperty float z\n")
    f.write("property uchar red\nproperty uchar green\nproperty uchar blue\n")
    f.write("end_header\n")
    for (x, y, z), (r, g, b) in zip(pts3d, colors):
        f.write(f"{x} {y} {z} {int(r)} {int(g)} {int(b)}\n")

print("Saved:", ply_path)


In [ ]:
#Download colored PLY
from google.colab import files
files.download("/content/mast3r/output/scene_color.ply")


In [ ]:
pip install open3d


In [ ]:
# ONE-CELL INTERACTIVE PLY VIEWER (rotate + zoom) WITH ROBUST PARSER
#One-cell robust interactive PLY viewer
!pip install -q plotly

from google.colab import files  # if not in Colab, replace with your own file loading
import numpy as np
import plotly.graph_objects as go

# 1) upload the PLY file (pick your scene_color.ply)
uploaded = files.upload()

if not uploaded:
    print("No file uploaded.")
else:
    fname = next(iter(uploaded.keys()))
    print("Loaded file:", fname)

    points = []
    colors = []

    with open(fname, "r") as f:
        # skip header
        for line in f:
            if line.strip() == "end_header":
                break

        # parse data lines
        for line in f:
            line = line.strip()
            if not line:
                continue

            # clean weird chars: brackets, commas
            clean = line.replace("[", " ").replace("]", " ").replace(",", " ")
            parts = clean.split()
            # need at least x y z
            if len(parts) < 3:
                continue

            vals = list(map(float, parts[:6]))  # up to x y z r g b
            x, y, z = vals[0], vals[1], vals[2]
            points.append([x, y, z])

            if len(vals) >= 6:
                r, g, b = vals[3], vals[4], vals[5]
                # clamp 0–255
                r = max(0, min(255, r))
                g = max(0, min(255, g))
                b = max(0, min(255, b))
                colors.append([r, g, b])

    if not points:
        print("No points parsed from file.")
    else:
        pts = np.array(points)
        x, y, z = pts[:, 0], pts[:, 1], pts[:, 2]

        if colors and len(colors) == len(points):
            cols = np.array(colors, dtype=np.float32)
            # Plotly expects 0–255 in rgb() strings or 0–1 floats; we’ll use strings
            col_strings = [f"rgb({int(r)},{int(g)},{int(b)})" for r, g, b in cols]
            marker_kwargs = dict(size=2, color=col_strings)
        else:
            marker_kwargs = dict(size=2)

        fig = go.Figure(
            data=[
                go.Scatter3d(
                    x=x,
                    y=y,
                    z=z,
                    mode="markers",
                    marker=marker_kwargs,
                )
            ]
        )
        fig.update_layout(
            width=800,
            height=800,
            scene=dict(
                xaxis_title="X",
                yaxis_title="Y",
                zaxis_title="Z",
                aspectmode="data",
            ),
            title=f"{fname}",
        )
        fig.show()
